In [5]:
import spacy
import string

NER = spacy.load("en_core_web_sm")


In [30]:
sample1 = 'House # C 38, Block 8 ,Gulshan-e-Iqbal, Karachi'

def ner(textstring):
    text1= NER(textstring)

    for word in text1.ents:
        print(word.text,word.label_)

ner(sample1)

House ORG
38 CARDINAL
Gulshan GPE
Karachi PERSON


In [19]:
sample2 ='Hi, my name is Haider Abbas. I study in class III-B in Al Murtaza School. My university is IBA. I want to be a billionare when I grow up. I live in House  # C-38,  Block  8 , Gulshan-e-Iqbal, Karachi'

ner(sample2)

Haider Abbas PERSON
Al Murtaza School ORG
IBA ORG
House ORG
C-38 PRODUCT
Gulshan GPE
Karachi PERSON


In [18]:
sample3 = 'Apple is an American tech company whose headquarters are located in Cupertino, California. It was founded by Steve Jobs and Steve Wozniak in April 1976.'

ner(sample3)

Apple ORG
American NORP
Cupertino GPE
California GPE
Steve Jobs PERSON
Steve Wozniak PERSON
April 1976 DATE


In [1]:
# import nltk
from nltk.metrics.distance import edit_distance

In [29]:
correct_words = ['gulshan e hadeed', 'gulshan e iqbal', 'defence', 'clifton', 'meher plaza', 'al murtaza heights', 'al murtaza height']

incorrect_words= ['gulshen iqbal', 'defnse', 'klifton', 'mehar plaza', 'al murteza heights']

for word in incorrect_words:
		temp = [(edit_distance(word, w),w) for w in correct_words]
		# print(sorted(temp, key = lambda val:val[0])[0][1])

		if len(temp) > 1: 
			temp = [min(temp, key=lambda t: t[0])]

		print(temp)
		# print(temp[0][1])

# print(edit_distance(incorrect_words[2], correct_words[3]), correct_words[3])

[(3, 'gulshan e iqbal')]
[(2, 'defence')]
[(1, 'clifton')]
[(1, 'meher plaza')]
[(1, 'al murtaza heights')]


In [4]:
import pandas as pd

In [5]:
def load_corpus(file_name, pandas=False):
    if pandas==True:
        df = pd.read_csv(file_name, sep="\t", header=None, names=["Neighborhood"])
        return df
    else:
        with open(file_name, "r") as file:
            neighborhood_names = file.readlines()    
        refined_name_list = [name.strip() for name in neighborhood_names]
        return refined_name_list


def lowercase_conversion(text_str):
    return text_str.lower()


corpus_data = load_corpus("karachi_neighbourhoods.txt")

file_name = "karachi_neighbourhoods_lowered.txt"

with open(file_name, "w") as file:
    for item in corpus_data:
        item = lowercase_conversion(item)
        file.write(item + "\n")

new_corpus = load_corpus("karachi_neighbourhoods_lowered.txt", True)

print(new_corpus.head())

print(f"File '{file_name}' has been created.")



        Neighborhood
0         abbas town
1     abbasi shaheed
2  abdul rehman goth
3      abdullah goth
4           abidabad
File 'karachi_neighbourhoods_lowered.txt' has been created.


In [23]:
abbreviations_1 = {
    "Apt": "Apartment",
    "Bldg.": "Building",
    "Cantt.": "Cantonment",
    "Col.": "Colony",
    "Ext.": "Extension",
    "Hno.": "House Number",
    "Mkt.": "Market",
    "Ngr.": "Nagar",
    "P.O.": "Post Office",
    "P.O.B": "Post Office Box",
    "Pl.": "Place",
    "Sec.": "Sector",
    "Twn.": "Town",
    "Vlg.": "Village",
    "Dist.": "District",
    "Rd.": "Road",
    "Jct.": "Junction",
    "Ln.": "Lane",
    "Masjid": "Mosque",
    "Fl.": "Floor",
    "Blk.": "Block",
    "Sch.": "School",
    "Univ.": "University",
    "Corp.": "Corporation",
    "Ctr.": "Center",
    "Plz.": "Plaza",
    "Ph.": "Phase",
    "G.P.O.": "General Post Office",
    "Bzr.": "Bazaar",
    "Est.": "Estate",
    # Add more mappings as needed
}


abbreviations_2 = {
    "St.": "Street",
    "Ave": "Avenue",
    "Rd.": "Road",
    "Blvd": "Boulevard",
    "Plz.": "Plaza",
    "Pkwy": "Parkway",
    "Hwy": "Highway",
    "Sec.": "Sector",
    "Apt": "Apartment",
    "Bldg.": "Building",
    "P.O.": "Post Office",
    "P.O.B": "Post Office Box",
    "G.P.O.": "General Post Office",
    "Jct.": "Junction",
    "Univ.": "University",
    "Est.": "Estate",
    "Cantt.": "Cantonment",
    "Masjid": "Mosque",
    "Res.": "Residential",
    "Col.": "Colony",
    "Ext.": "Extension",
    "Phase": "Phase",
    "Ngr.": "Nagar",
    "Bzr.": "Bazaar",
    # Add more mappings as needed
}

print(len(abbreviations_1))
print(len(abbreviations_2))

30
24


In [18]:
def compare_dictionaries(dict1, dict2):
    dict1_keys = set(dict1.keys())
    dict2_keys = set(dict2.keys())

    dict1_missing_keys = dict1_keys - dict2_keys
    dict2_missing_keys = dict2_keys - dict1_keys

    return dict1_missing_keys, dict2_missing_keys


# Compare the dictionaries
missing_keys_dict1, missing_keys_dict2 = compare_dictionaries(abbreviations_1, abbreviations_2)

# Print the missing keys
print("Missing keys in dict1 from dict2:", missing_keys_dict1)
print("Missing keys in dict2 from dict1:", missing_keys_dict2)


for x in abbreviations_1:
    if x not in abbreviations_2:
        abbreviations_2[x] = abbreviations_1[x]

missing_keys_dict1, missing_keys_dict2 = compare_dictionaries(abbreviations_1, abbreviations_2)

# Print the missing keys
print("Missing keys in dict1 from dict2:", missing_keys_dict1)
print("Missing keys in dict2 from dict1:", missing_keys_dict2)

print(abbreviations_1)


Missing keys in dict1 from dict2: {'Blk.', 'Vlg.', 'Ctr.', 'Twn.', 'Pl.', 'Ph.', 'Hno.', 'Ln.', 'Sch.', 'Fl.', 'Mkt.', 'Corp.', 'Dist.'}
Missing keys in dict2 from dict1: {'Phase', 'Pkwy', 'Blvd', 'Ave', 'Hwy', 'Res.', 'St.'}
Missing keys in dict1 from dict2: set()
Missing keys in dict2 from dict1: {'Phase', 'Pkwy', 'Blvd', 'Ave', 'Hwy', 'Res.', 'St.'}
{'Apt': 'Apartment', 'Bldg.': 'Building', 'Cantt.': 'Cantonment', 'Col.': 'Colony', 'Ext.': 'Extension', 'Hno.': 'House Number', 'Mkt.': 'Market', 'Ngr.': 'Nagar', 'P.O.': 'Post Office', 'P.O.B': 'Post Office Box', 'Pl.': 'Place', 'Sec.': 'Sector', 'Twn.': 'Town', 'Vlg.': 'Village', 'Dist.': 'District', 'Rd.': 'Road', 'Jct.': 'Junction', 'Ln.': 'Lane', 'Masjid': 'Mosque', 'Fl.': 'Floor', 'Blk.': 'Block', 'Sch.': 'School', 'Univ.': 'University', 'Corp.': 'Corporation', 'Ctr.': 'Center', 'Plz.': 'Plaza', 'Ph.': 'Phase', 'G.P.O.': 'General Post Office', 'Bzr.': 'Bazaar', 'Est.': 'Estate'}


In [22]:
print(len(abbreviations_1))
print(len(abbreviations_2))

30
37


In [24]:
def combine_dictionaries(dict1, dict2):
    combined_dict = dict1.copy()  # Create a copy of the first dictionary
    combined_dict.update(dict2)  # Update the copy with the second dictionary

    return combined_dict

combined_dict = combine_dictionaries(abbreviations_1, abbreviations_2)

# Print the combined dictionary

print(len(combined_dict))

print(combined_dict)

37
{'Apt': 'Apartment', 'Bldg.': 'Building', 'Cantt.': 'Cantonment', 'Col.': 'Colony', 'Ext.': 'Extension', 'Hno.': 'House Number', 'Mkt.': 'Market', 'Ngr.': 'Nagar', 'P.O.': 'Post Office', 'P.O.B': 'Post Office Box', 'Pl.': 'Place', 'Sec.': 'Sector', 'Twn.': 'Town', 'Vlg.': 'Village', 'Dist.': 'District', 'Rd.': 'Road', 'Jct.': 'Junction', 'Ln.': 'Lane', 'Masjid': 'Mosque', 'Fl.': 'Floor', 'Blk.': 'Block', 'Sch.': 'School', 'Univ.': 'University', 'Corp.': 'Corporation', 'Ctr.': 'Center', 'Plz.': 'Plaza', 'Ph.': 'Phase', 'G.P.O.': 'General Post Office', 'Bzr.': 'Bazaar', 'Est.': 'Estate', 'St.': 'Street', 'Ave': 'Avenue', 'Blvd': 'Boulevard', 'Pkwy': 'Parkway', 'Hwy': 'Highway', 'Res.': 'Residential', 'Phase': 'Phase'}


In [28]:
missing_keys_dict1, missing_keys_dict2 = compare_dictionaries(abbreviations_1, combined_dict)
missing_keys_dict3, missing_keys_dict4 = compare_dictionaries(abbreviations_2, combined_dict)

# Print the missing keys
print("Missing keys in dict1 from combined_dict:", missing_keys_dict1)
print("Missing keys in combined_dict from dict1:", missing_keys_dict2)

print("Missing keys in dict3 from combined_dict:", missing_keys_dict3)
print("Missing keys in combined_dict from dict3:", missing_keys_dict4)

Missing keys in dict1 from combined_dict: set()
Missing keys in combined_dict from dict1: {'Phase', 'Pkwy', 'Blvd', 'Ave', 'Hwy', 'Res.', 'St.'}
Missing keys in dict3 from combined_dict: set()
Missing keys in combined_dict from dict3: {'Blk.', 'Vlg.', 'Ctr.', 'Twn.', 'Pl.', 'Ph.', 'Hno.', 'Ln.', 'Sch.', 'Fl.', 'Mkt.', 'Corp.', 'Dist.'}


In [30]:
print(combined_dict)

abbreviations = {
    'Apt': 'Apartment',
    'Bldg.': 'Building',
    'Cantt.': 'Cantonment',
    'Col.': 'Colony',
    'Ext.': 'Extension',
    'Hno.': 'House Number',
    'Mkt.': 'Market',
    'Ngr.': 'Nagar',
    'P.O.': 'Post Office',
    'P.O.B': 'Post Office Box',
    'Pl.': 'Place',
    'Sec.': 'Sector',
    'Twn.': 'Town',
    'Vlg.': 'Village',
    'Dist.': 'District',
    'Rd.': 'Road',
    'Jct.': 'Junction',
    'Ln.': 'Lane',
    'Masjid': 'Mosque',
    'Fl.': 'Floor',
    'Blk.': 'Block',
    'Sch.': 'School',
    'Univ.': 'University',
    'Corp.': 'Corporation',
    'Ctr.': 'Center',
    'Plz.': 'Plaza',
    'Ph.': 'Phase',
    'G.P.O.': 'General Post Office',
    'Bzr.': 'Bazaar',
    'Est.': 'Estate',
    'St.': 'Street',
    'Ave': 'Avenue',
    'Blvd':
    'Boulevard',
    'Pkwy': 'Parkway',
    'Hwy': 'Highway',
    'Res.': 'Residential',
    'Phase': 'Phase'
    }

if abbreviations == combined_dict:
    print(True)
else:
    print(False)

{'Apt': 'Apartment', 'Bldg.': 'Building', 'Cantt.': 'Cantonment', 'Col.': 'Colony', 'Ext.': 'Extension', 'Hno.': 'House Number', 'Mkt.': 'Market', 'Ngr.': 'Nagar', 'P.O.': 'Post Office', 'P.O.B': 'Post Office Box', 'Pl.': 'Place', 'Sec.': 'Sector', 'Twn.': 'Town', 'Vlg.': 'Village', 'Dist.': 'District', 'Rd.': 'Road', 'Jct.': 'Junction', 'Ln.': 'Lane', 'Masjid': 'Mosque', 'Fl.': 'Floor', 'Blk.': 'Block', 'Sch.': 'School', 'Univ.': 'University', 'Corp.': 'Corporation', 'Ctr.': 'Center', 'Plz.': 'Plaza', 'Ph.': 'Phase', 'G.P.O.': 'General Post Office', 'Bzr.': 'Bazaar', 'Est.': 'Estate', 'St.': 'Street', 'Ave': 'Avenue', 'Blvd': 'Boulevard', 'Pkwy': 'Parkway', 'Hwy': 'Highway', 'Res.': 'Residential', 'Phase': 'Phase'}
True


punctuation removing: (from sources)
- city areas corpus (only has dashes -)
- abbriviations
- data to process

